# Finetune a T5 Transformer model on the California BillSum dataset

In [1]:
!pip install -q transformers datasets evaluate rouge_score accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


# Load BillSum Dataset from the Datasets library

BillSum: summarization of US Congressional and California state bills.



In [2]:
from datasets import load_dataset

billsum = load_dataset("billsum", split = "ca_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [3]:
# Split the dataset into a train and test set
billsum = billsum.train_test_split(test_size=0.2)

In [4]:
# Look at the data
print(billsum)

# The text column is our model input
# The summary column is our model target
print("ORIGINAL TEXT:\n", billsum['train']['text'][0], "\n")
print("SUMMARY:\n", billsum['train']['summary'][0], "\n")

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 248
    })
})
ORIGINAL TEXT:
 The people of the State of California do enact as follows:


SECTION 1.
Section 166 of the Penal Code is amended to read:
166.
(a) Except as provided in subdivisions (b), (c), and (d), a person guilty of any of the following contempts of court is guilty of a misdemeanor:
(1) Disorderly, contemptuous, or insolent behavior committed during the sitting of a court of justice, in the immediate view and presence of the court, and directly tending to interrupt its proceedings or to impair the respect due to its authority.
(2) Behavior specified in paragraph (1) that is committed in the presence of a referee, while actually engaged in a trial or hearing, pursuant to the order of a court, or in the presence of any jury while actually sitting for the trial of a cause, or 

In [4]:
# Preprocessing
## Load Model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
# Check Model
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [7]:
# named_parameters is defined in torch.nn.Module, which is a superclass of T5ForConditionalGeneration
for name, param in model.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Requires gradients: {param.requires_grad}")
    print(f"Parameter shape: {param.shape}")
    print("=" * 30)

Parameter name: shared.weight
Requires gradients: True
Parameter shape: torch.Size([32128, 512])
Parameter name: encoder.block.0.layer.0.SelfAttention.q.weight
Requires gradients: True
Parameter shape: torch.Size([512, 512])
Parameter name: encoder.block.0.layer.0.SelfAttention.k.weight
Requires gradients: True
Parameter shape: torch.Size([512, 512])
Parameter name: encoder.block.0.layer.0.SelfAttention.v.weight
Requires gradients: True
Parameter shape: torch.Size([512, 512])
Parameter name: encoder.block.0.layer.0.SelfAttention.o.weight
Requires gradients: True
Parameter shape: torch.Size([512, 512])
Parameter name: encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight
Requires gradients: True
Parameter shape: torch.Size([32, 8])
Parameter name: encoder.block.0.layer.0.layer_norm.weight
Requires gradients: True
Parameter shape: torch.Size([512])
Parameter name: encoder.block.0.layer.1.DenseReluDense.wi.weight
Requires gradients: True
Parameter shape: torch.Size([2048, 5

In [ ]:
# # Optional: Specify layers to fine tune and which to freeze by setting requires grad to true and false
layers_to_fine_tune = ['classifier.weight','classifier.bias']

# Freeze layers
for name, param in model.named_parameters():
    if not any(layer_name in name for layer_name in layers_to_fine_tune):
        param.requires_grad = False

# Unfreeze Fine-tune layers
for name, param in model.named_parameters():
    if any(layer_name in name for layer_name in layers_to_fine_tune):
        param.requires_grad = True

# Preprocessing

We need to create a preprocess function that we will apply to every instance in the dataset. The preprocess function needs to:

1. Prefix input with a prompt so T5 knows it's performing a summarisation task.

2. Use the keyword `text_target` argument when tokenizing labels

3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter

After applying the preprocessing to every instance in the dataset, we create batches of examples using a data collator function. This is because model training must be in batches.

In [5]:
def preprocess_function(examples, prefix: str = "summarize: "):
  # prepend the prefix to every instance in the dataset
  inputs = [prefix + doc for doc in examples["text"]]

  # Tokenize the text input and convert them into tensors
  model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

  # Tokenize the labels and convert them into tensors
  labels = tokenizer(text_target=examples['summary'], max_length=123, truncation=True)

  model_inputs["labels"] = labels["input_ids"]

  return model_inputs

In [6]:
# Apply preprocessing over entire dataset - batched = True process multiple elements of the datasets
# Method `map` is directly defined in class `Dataset` (https://github.com/huggingface/datasets/blob/2.17.0/src/datasets/arrow_dataset.py#L2867)
tokenized_billsum = billsum.map(preprocess_function, batched = True,)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [10]:
tokenized_billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 248
    })
})

In [7]:
# Create a batch of examples, with dynamic padding. Use the appropriate collator function
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model_checkpoint,)

# Evaluate & Prepare DataLoaders
We require monitoring a metric during training to see how well our model is doing. We use the `evaluate` library to load an evaluation metric - we use the ROUGE metric.

With the loss metric defined, we must define a function that takes model predictions and labels and computes the loss metric. This is usually called the `compute_metrics` function.

In [8]:
tokenized_billsum_trch = tokenized_billsum.remove_columns(["text", "summary", "title"])

tokenized_billsum_trch.set_format("torch")
tokenized_billsum_trch

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 989
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 248
    })
})

In [9]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_billsum_trch['train'], shuffle = True, batch_size = 8, collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_billsum_trch['test'], shuffle = True, batch_size = 8, collate_fn=data_collator
)

In [24]:
# Peek at the training data batch
import pprint
first_batch = next(iter(train_dataloader))

#print(batch)
print(len(train_dataloader))
print(f"input_ids batch shape: {first_batch.input_ids.shape}")
print(f"attention_mask batch shape: {first_batch.attention_mask.shape}")
print(f"labels batch shape: {first_batch.labels.shape}")

248
input_ids batch shape: torch.Size([4, 1024])
attention_mask batch shape: torch.Size([4, 1024])
labels batch shape: torch.Size([4, 123])


## Use [rouge](https://huggingface.co/spaces/evaluate-metric/rouge) for evaluation

**From the documentation:** ROUGE, or Recall-Oriented Understudy for Gisting Evaluation, is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing.

We use rouge **with stemmer**, which uses Porter stemmer to strip word suffixes.


In [10]:
import evaluate

rouge = evaluate.load("rouge")

# Train Model using PyTorch

In [11]:
from transformers import AdamW, get_scheduler
import torch
import numpy as np

# Define Optimiser
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

# Define Loss Function
def compute_metrics(predictions, labels):

  # Convert token ids to tokens
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

  # If the label is -100 then it's a pad token and we assign that token id to <PAD>
  # The use of -100 as padding token id for labels does not seem to be well documented,
  # but it is mentioned in https://github.com/huggingface/transformers/issues/9770, which I found thanks to preplexity!
  labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

  # Count non padding tokens
  prediction_lens = [torch.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]

  result['gen_len'] = torch.stack(prediction_lens).float().mean()

  return {k: v for (k,v) in result.items()}

# Initialize variables to track the best model
best_loss = float('inf')
best_checkpoint_path = None

# Collect Statistics
train_loss = []
train_metrics = []
test_metrics = []

## Place training on a GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
# Modify the model (which is a module) in-place, to match the device.
# API documentation: https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.to
model.to(device)

# Define Learning Rate Scheduler
num_epochs = 4
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

cpu


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Set up a list to store checkpoints
checkpoint_paths = []

# Set the module (i.e., the model) in training mode.
model.train()

# Training loop
for epoch in range(num_epochs):
    for i, batch in enumerate(train_dataloader):
        # Set Gradients to 0
        optimizer.zero_grad()

        # Perform a forward model pass
        ## Put the batch onto a GPU
        batch = {k: v.to(device) for (k, v) in batch.items()}

        ## Forward Pass
        outputs = model(**batch)

        # Compute Loss
        loss = outputs.loss

        # Compute Metric
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        labels = batch["labels"]

        metrics = compute_metrics(predictions=predictions, labels=labels)

        # Store Metrics
        train_loss.append(float(loss))
        train_metrics.append(metrics)

        # Backward pass to update parameters
        ## Compute fradients with respect to model parameters
        loss.backward()

        # Optimizer step
        ## Use the computed gradients to update the model parameters - adjust parameters in the direction that reduces the loss
        optimizer.step()

        # Update Learning Rate - according to a schedule. This adjusts learning rate dynamically
        lr_scheduler.step()

        # Print Progress
        print(f"epoch {epoch} batch_number {i} loss {loss} metrics {metrics}")

    # Save checkpoint at certain intervals
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'metrics': metrics,
        # Add other relevant information if needed
    }
    checkpoint_path = f'checkpoint_epoch_{epoch}_batch_{i}.bin'
    torch.save(checkpoint, checkpoint_path)
    checkpoint_paths.append(checkpoint_path)

    # Update best_loss and best_checkpoint_path if needed
    if loss < best_loss:
        best_loss = loss
        best_checkpoint_path = checkpoint_path




You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


epoch 0 batch_number 0 loss 4.721962928771973 metrics {'rouge1': 0.46809404247214925, 'rouge2': 0.13214741380819514, 'rougeL': 0.3660475944003177, 'rougeLsum': 0.3654987212020808, 'gen_len': tensor(123.)}


In [ ]:
# Load the best model checkpoint
best_checkpoint = torch.load(best_checkpoint_path)

model.load_state_dict(best_checkpoint['model_state_dict'])
optimizer.load_state_dict(best_checkpoint['optimizer_state_dict'])

best_epoch = best_checkpoint['epoch']
best_loss = best_checkpoint['loss']
best_metrics = best_checkpoint['metrics']


In [ ]:
print(best_epoch)
print(best_loss)
print(best_metrics)

In [ ]:
model.eval()

eval_metrics = []
for epoch in range(num_epochs):
  for i, batch in enumerate(test_dataloader):
    # Perform a forward model pass
    ## Put the batch onto a GPU
    batch = {k: v.to(device) for (k, v) in batch.items()}

    ## Forward Pass - Set no grad because we don't want to update parameters in validation
    with torch.no_grad():
        outputs = model(**batch)

    # Compute Metric
    logits = outputs.logits
    predictions = torch.argmax(logits, dim = -1)
    labels = batch["labels"]

    metrics = compute_metrics(predictions=predictions, labels=labels)

    # Store Metrics
    eval_metrics.append(metrics)

    # Print Progress
    print(f"epoch {epoch} batch_number {i} metrics {metrics}")


# Save and Load Tokenizer and Model

In [ ]:
# Suggested from Docs: https://huggingface.co/transformers/v1.2.0/serialization.html
# Save Tokenizer and Model
import os

output_dir = "./billsum_summariser_model/"
os.makedirs(output_dir, exist_ok=True)

# Step 1: Save a model, configuration and vocabulary that you have fine-tuned

# If we have a distributed model, save only the encapsulated model
# (it was wrapped in PyTorch DistributedDataParallel or DataParallel)
model_to_save = model.module if hasattr(model, 'module') else model

# If we save using the predefined names, we can load using `from_pretrained`
WEIGHTS_NAME = "pytorch_model.bin"
CONFIG_NAME = "config.json"
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)

torch.save(model.state_dict(), output_model_file)
model.config.to_json_file(output_config_file)
tokenizer.save_pretrained(output_dir)

In [ ]:
# Load Model
pretrained_loaded_model = AutoModelForSeq2SeqLM.from_pretrained("billsum_summariser_model")

# Inference

In [ ]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [ ]:
# Inference
## Tokenize inputs
inputs = tokenizer(text, return_tensors="pt").input_ids
print(inputs)

In [ ]:
outputs = pretrained_loaded_model.generate(inputs,
                                           max_new_tokens=35,
                                           do_sample=False,
                                           )
print(outputs)

In [ ]:
import pprint
original_text = {"original_text": text}
summary_text = {"summary_text": tokenizer.decode(outputs[0],
                                                 skip_special_tokens=True
                                                 )}

pprint.pprint(original_text)
pprint.pprint(summary_text)